In [1]:
import warnings
warnings.filterwarnings('ignore')

import pyspark.sql
from pyspark.sql.functions import *

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from wrangle import wrangle_311

spark = pyspark.sql.SparkSession.builder.getOrCreate()

df = wrangle_311(spark)
print("\ndf shape: (%d, %d)\n" % (df.count(), len(df.columns)))
df.show(1, vertical=True)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/09 15:44:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[wrangle.py] reading case.csv


[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features


[wrangle.py] joining departments



df shape: (841704, 20)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only show

Answer the questions below by using a combination of the techniques discussed in the lesson that you think is appropriate.

1. How many different cases are there, by department?

In [2]:
df.groupby('department').count().show()

+--------------------+------+
|          department| count|
+--------------------+------+
|         Solid Waste|279270|
|Animal Care Services|116915|
|Trans & Cap Impro...| 96193|
|  Parks & Recreation| 19907|
|    Customer Service|  2849|
|        Metro Health|  5163|
|        City Council|    33|
|DSD/Code Enforcement|321374|
+--------------------+------+



2. Does the percentage of cases that are late vary by department?

In [3]:
nrows = df.count()
(
    df.groupby(['department', 'case_late'])
    .count()
    .withColumn('percentage', col('count') / nrows)
    .filter(df.case_late == 'true')
    .show()
)

+--------------------+---------+-----+--------------------+
|          department|case_late|count|          percentage|
+--------------------+---------+-----+--------------------+
|    Customer Service|     true| 2010|0.002388012888141199|
|Trans & Cap Impro...|     true| 5411|0.006428625740165189|
|        Metro Health|     true|  829|9.849068080940568E-4|
|Animal Care Services|     true|23276| 0.02765342685789779|
|  Parks & Recreation|     true| 3797| 0.00451108703297121|
|         Solid Waste|     true|32945| 0.03914083810935911|
|DSD/Code Enforcement|     true|26235|0.031168914487753413|
+--------------------+---------+-----+--------------------+



3. On average, how late are the late cases by department?

4. What is the service type that is the most late? Just for Parks & Rec?

In [4]:
df.filter(df.case_late == 'true').groupby('department').agg(mean(df.num_days_late).alias('average_late')).show()

+--------------------+------------------+
|          department|      average_late|
+--------------------+------------------+
|         Solid Waste| 7.186821906120899|
|Animal Care Services|23.458633245820124|
|Trans & Cap Impro...|10.603064680316946|
|  Parks & Recreation|22.348910457867518|
|    Customer Service| 87.68385942150394|
|        Metro Health|6.5438133155476494|
|DSD/Code Enforcement| 49.38428705358908|
+--------------------+------------------+



5. For the DSD/Code Enforcement department, what are the most common service request types? Look at other departments too.

In [5]:
(
    df.filter(df.department == 'DSD/Code Enforcement')
    .groupby('service_request_type')
    .count()
    .sort(col('count').desc())
    .show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Overgrown Yard/Trash|65895|
|        Bandit Signs|32910|
|Front Or Side Yar...|28794|
|Junk Vehicle On P...|21473|
|Alley-Way Mainten...|20214|
|Right Of Way/Side...|17699|
|Vacant Lot/Overgr...| 9329|
|Minimum Housing-O...| 8543|
|Graffiti: Private...| 8525|
|Dang. Premises/CL...| 8004|
|Minimum Housing-T...| 7984|
|"Vacant Lot-Priva...| 7875|
|    Brush Violations| 7475|
|    Permits Building| 6880|
|Vacant House Over...| 6109|
|  Graffiti: Building| 4572|
|Graffiti Public P...| 4104|
|Low Hanging Limbs...| 4052|
|CPS Energy Wood P...| 3501|
|Zoning Business i...| 2716|
+--------------------+-----+
only showing top 20 rows



6. Does whether or not its a weekend matter for when a case is opened/closed?

In [6]:
fmt = "M/d/yy"

open_close = (
    df.select('case_opened_date', 'case_closed_date')
    .withColumn('open_date', to_date('case_opened_date'))
    .withColumn('close_date', to_date('case_closed_date'))
    .drop('case_opened_date').drop('case_closed_date')
)

opened_weekend_count = (
    open_close.withColumn("opened_weekend", 
                          date_format("open_date", 'EEE')
                          .isin(["Sat", "Sun"]))
    .groupby('opened_weekend')
    .count()
)

closed_weekend_count = (
    
    open_close.withColumn("closed_weekend", 
                          date_format("close_date", 'EEE')
                          .isin(["Sat", "Sun"]))
    .groupby('closed_weekend')
    .count()
)

In [7]:
opened_weekend_count.show()

+--------------+------+
|opened_weekend| count|
+--------------+------+
|          true| 82443|
|         false|759261|
+--------------+------+



In [8]:
closed_weekend_count.show()

+--------------+------+
|closed_weekend| count|
+--------------+------+
|          true| 88027|
|         false|735567|
|          null| 18110|
+--------------+------+



7. On average, how many cases are opened a day for the Customer Service department?

In [9]:
(
    df.filter(df.department == 'Customer Service')
    .withColumn('open_date', to_date('case_opened_date'))
    .groupby(['department', 'open_date'])
    .count()
    .agg({'count':'mean'})
    .show()
)

+-----------+
| avg(count)|
+-----------+
|5.564453125|
+-----------+



8. Does the number of service requests for the solid waste department vary by day of the week?

In [10]:
(
    df.filter(df.department == 'Solid Waste')
    .withColumn('weekday', date_format("case_opened_date", 'EEE'))
    .groupby(['department', 'weekday'])
    .count()
    .show()
)

+-----------+-------+-----+
| department|weekday|count|
+-----------+-------+-----+
|Solid Waste|    Wed|41947|
|Solid Waste|    Thu|47718|
|Solid Waste|    Sun| 6885|
|Solid Waste|    Mon|58085|
|Solid Waste|    Sat|15312|
|Solid Waste|    Fri|51203|
|Solid Waste|    Tue|58120|
+-----------+-------+-----+

